# Setting up your own custom task

If the tasks we have set up don't fit your use case, this guide will go into how to set up your own task from scratch.
To do this, we will be setting up a simple keyword extraction task.

Keyword extraction is blabla.
An example use case could be blabla.
A full implementation can be found in blabla.

Let's start with the task interface.
The full Task interface can be found in [task.py](task.py).
However, to implement a Task there are only a few parts relevant to us.
The simplified interface for this guide is basically:

```python
Input = TypeVar("Input", bound=PydanticSerializable)
Output = TypeVar("Output", bound=PydanticSerializable)

class Task(ABC, Generic[Input, Output]):
    @abstractmethod
    def run(self, input: Input, logger: DebugLogger) -> Output:
        """Executes the process for this use-case."""
        ...
```

To create our own task, we have to define our Input, Output and how we would like to run it.
Since tasks can vary so much, no assumptions are done about the implementation of the task. 
The only requirement is the fact that the input and output have to be PydanticSerializable.
For our keyword extraction our input and output will be the following:

In [3]:
from typing import Sequence
from pydantic import BaseModel


class KeywordExtractionInput(BaseModel):
    text: str

class KeywordExtractionOutput(BaseModel):
    keywords: Sequence[str]

Now that we have our input and output defined, we can make the task: 

In [ ]:
from intelligence_layer.task import DebugLogger, Task

class KeywordExtractionTask(Task[KeywordExtractionInput, KeywordExtractionOutput]):
    def run(self, input: KeywordExtractionInput, logger: DebugLogger) -> KeywordExtractionOutput:
        pass